Zero Indegree Inits
===

Notebook exploring: how many inits are with zero-indegree sites?

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import os
import re
import json
import sys
import pickle
from tqdm import tqdm

import sklearn
import sklearn.linear_model
import sklearn.preprocessing
from sklearn.pipeline import Pipeline

import dateutil.parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import pytz

In [ ]:
# HuggingFace packages
import transformers
import tokenizers
import torch

# more torch imports
import torchvision
import torchvision.transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# evaluation
from scipy.stats import rankdata

In [ ]:
from pathlib import Path
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = Path(git_root_dir[0].strip())
git_root_dir

In [ ]:
import sys
sys.path.append(os.path.join(git_root_dir, 'src'))
import cbrec.genconfig

In [ ]:
config = cbrec.genconfig.Config()
#config.metadata_filepath += "_old"
#config.feature_db_filepath += "_old"

In [ ]:
import cbrec.featuredb
import cbrec.utils
import cbrec.data
import cbrec.reccontext
import cbrec.evaluation
import cbrec.torchmodel
import cbrec.text.embeddingdb
import cbrec.text.journalid

In [ ]:
import cbrec.logutils
cbrec.logutils.set_up_logging()

In [ ]:
# turn off matplotlib logging
import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

In [ ]:
import sys
caringbridge_core_path = "/home/lana/levon003/repos/caringbridge_core"
sys.path.append(caringbridge_core_path)
import cbcore.data.paths

## Stream reccontexts

Score all test_contexts (including test and predict contexts) with the baselines.

In [ ]:
try:
    import cbrec
except:
    sys.path.append("/home/lana/levon003/repos/recsys-peer-match/src")

import cbrec.featuredb
import cbrec.genconfig
import cbrec.utils
import cbrec.evaluation
import cbrec.reccontext
import cbrec.recentActivityCounter
import cbrec.modeling.text_loader
import cbrec.modeling.reccontext_builder
import cbrec.modeling.scorer
import cbrec.modeling.manager

In [ ]:
VALIDATION_END_TIMESTAMP = datetime.strptime("2021-07-01", "%Y-%m-%d").timestamp() * 1000
test_md_list = [md for md in cbrec.utils.stream_metadata_list(config.metadata_filepath) if md['type'] == 'test' and md['timestamp'] <= VALIDATION_END_TIMESTAMP]
test_md_map = {md['metadata_id']: md for md in test_md_list}
len(test_md_map)

In [ ]:
config = cbrec.genconfig.Config()

data = []
db = cbrec.featuredb.get_db_by_filepath(config.feature_db_filepath)
with db:
    for test_context in tqdm(cbrec.featuredb.stream_test_contexts(db, config), desc='Streaming test contexts', total=len(test_md_map)):
        if test_context['metadata_id'] not in test_md_map:
            continue
        test_context_md = test_md_map[test_context['metadata_id']]
        #if test_context_md['timestamp'] <= VALIDATION_END_TIMESTAMP:
        #    continue

        rc = cbrec.reccontext.RecContext.create_from_test_context(config, test_context_md, test_context)
        has_target = rc.md['has_target']
        assert has_target
        
        test_context_md['indegree'] = rc.candidate_usp_mat[rc.target_inds, 0].min()
        test_context_md['outdegree'] = rc.candidate_usp_mat[rc.target_inds, 1].min()
        data.append(test_context_md)
len(data)

In [ ]:
df = pd.DataFrame(data)
df.head()

In [ ]:
df['zero_indegree'] = df.indegree == 0
df.zero_indegree.value_counts()

In [ ]:
len(df)

In [ ]:
df.zero_indegree.sum() / len(df)

In [ ]:
df['zero_outdegree'] = df.outdegree == 0
df.zero_outdegree.sum() / len(df)